In [1]:
import numpy as np 
import pandas as pd
from sklearn import datasets 
from sklearn.metrics import accuracy_score
from sklearn import linear_model as lm 
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer 
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score 
from sklearn.model_selection import train_test_split
import re
from datetime import datetime


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
fire6 = pd.read_csv("data_cleaned.csv")

FileNotFoundError: ignored

In [ ]:
len(fire6)

1476

In [ ]:
np.average(fire6["MajorIncident"])

0.2696476964769648

In [ ]:
fire6.rename(columns={"Days Elapsed": "DaysElapsed", "Average Temp F": "AverageTempF"})
fire6.head()

,Latitude,Longitude,Started,Extinguished,Days Elapsed,AcresBurned,MajorIncident,Average Temp F,uvIndex,sunHour,totalSnow_cm,AirTankers,CalFireIncident,Counties,CrewsInvolved,Dozers,Engines,Fatalities,Helicopters,Injuries,PersonnelInvolved,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,WaterTenders,DaysElapsed,AverageTempF
0,37.86,-120.09,2013-08-17,2013-09-06,20,257314.0,False,73.0,7,13.3,0.0,0.0,True,Tuolumne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,73.0
1,34.59,-118.42,2013-05-30,2013-06-08,9,30274.0,False,70.5,7,14.5,0.0,0.0,True,Los Angeles,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,70.5
2,33.71,-116.73,2013-07-15,2013-07-30,15,27531.0,False,77.0,7,14.1,0.0,0.0,True,Riverside,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,77.0
3,39.12,-120.65,2013-08-10,2013-08-30,20,27440.0,False,60.5,6,13.9,0.0,0.0,False,Placer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,60.5
4,37.28,-119.32,2013-07-22,2013-09-24,64,22992.0,False,67.5,6,14.5,0.0,0.0,False,Fresno,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64,67.5


In [ ]:
Xc = fire6.drop(columns = ["MajorIncident"])
yc = fire6[["MajorIncident"]]

Xr = fire6.drop(columns = ["DaysElapsed", "AcresBurned"])
yrd = fire6[["DaysElapsed"]]
yrb = fire6[["AcresBurned"]]

In [ ]:
#classification train test split 
Xc_tr, Xc_te, yc_tr, yc_te = train_test_split(Xc, yc, test_size=0.2, random_state=42)

#regression for days elapsed train test split
Xrd_tr, Xrd_te, yrd_tr, yrd_te = train_test_split(Xr, yrd, test_size=0.2, random_state=42)

#regression for acres burned train test split
Xrb_tr, Xrb_te, yrb_tr, yrb_te = train_test_split(Xr, yrb, test_size=0.2, random_state=42)

In [ ]:
indices = Xc_te.index
indices

Int64Index([1125,  184, 1226,   67,  220,  903,  430,  240,  218,   49,
            ...
             999,  746,  394,  943,  561,  950,  997, 1131, 1159, 1279],
           dtype='int64', length=296)

In [ ]:
train = Xc_tr
train["MajorIncident"] = yc_tr
test = Xc_te
test["MajorIncident"] = yc_te

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def VIF(df, col):
    vals = sm.add_constant(df[col]).values
    num_col = len(col) + 1
    vif = [variance_inflation_factor(vals, i) for i in range(num_col)]
    return pd.Series(vif[1:], index = col)

In [ ]:
lrd1 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + uvIndex + sunHour + totalSnow_cm + AirTankers + CrewsInvolved + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresEvacuated + StructuresThreatened + WaterTenders',data = train).fit()
print(lrd1.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     6.459
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           3.24e-16
Time:                        08:16:22   Log-Likelihood:                -6938.3
No. Observations:                1180   AIC:                         1.392e+04
Df Residuals:                    1160   BIC:                         1.402e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -29.77

In [ ]:
#removed airtankers
lrd2 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + uvIndex + sunHour + totalSnow_cm + CrewsInvolved + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresEvacuated + StructuresThreatened + WaterTenders',data = train).fit()
print(lrd2.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     6.803
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           1.41e-16
Time:                        08:16:23   Log-Likelihood:                -6938.5
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1161   BIC:                         1.401e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -32.39

In [ ]:
#removed structures evacuated
lrd3 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + uvIndex + sunHour + totalSnow_cm + CrewsInvolved + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened + WaterTenders',data = train).fit()
print(lrd3.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     6.803
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           1.41e-16
Time:                        08:16:24   Log-Likelihood:                -6938.5
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1161   BIC:                         1.401e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -32.39

In [ ]:
#removed total snow
lrd4 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + uvIndex + sunHour + CrewsInvolved + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened + WaterTenders',data = train).fit()
print(lrd4.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     7.194
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           5.69e-17
Time:                        08:16:24   Log-Likelihood:                -6938.6
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1162   BIC:                         1.400e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -31.39

In [ ]:
#removed uv index
lrd5 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + sunHour + CrewsInvolved + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened + WaterTenders',data = train).fit()
print(lrd5.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     7.648
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           2.03e-17
Time:                        08:16:25   Log-Likelihood:                -6938.6
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1163   BIC:                         1.400e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -28.20

In [ ]:
#removed helicopters
lrd6 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + sunHour + CrewsInvolved + Dozers + Engines + Fatalities + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened + WaterTenders',data = train).fit()
print(lrd6.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     8.147
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           7.73e-18
Time:                        08:16:26   Log-Likelihood:                -6938.7
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1164   BIC:                         1.399e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -29.28

In [ ]:
#removed dozers
lrd7 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + sunHour + CrewsInvolved + Engines + Fatalities + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened + WaterTenders',data = train).fit()
print(lrd7.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     8.688
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           3.35e-18
Time:                        08:16:26   Log-Likelihood:                -6939.0
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1165   BIC:                         1.398e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -30.55

In [ ]:
#removed personnel
lrd7 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + sunHour + CrewsInvolved + Engines + Fatalities + Injuries + StructuresDamaged + StructuresDestroyed + StructuresThreatened + WaterTenders',data = train).fit()
print(lrd7.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     9.285
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           1.63e-18
Time:                        08:16:27   Log-Likelihood:                -6939.5
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1166   BIC:                         1.398e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -28.42

In [ ]:
#removed structures threatened
lrd8 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + sunHour + CrewsInvolved + Engines + Fatalities + Injuries + StructuresDamaged + StructuresDestroyed + WaterTenders',data = train).fit()
print(lrd8.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     9.687
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           3.33e-18
Time:                        08:16:27   Log-Likelihood:                -6941.6
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1167   BIC:                         1.398e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -35.93

In [ ]:
#removed watertenders
lrd9 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + AverageTempF + sunHour + CrewsInvolved + Engines + Fatalities + Injuries + StructuresDamaged + StructuresDestroyed',data = train).fit()
print(lrd9.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     10.49
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           1.49e-18
Time:                        08:16:28   Log-Likelihood:                -6942.1
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1168   BIC:                         1.397e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -32.46

In [ ]:
#removed avg temp
lrd10 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + MajorIncident + sunHour + CrewsInvolved + Engines + Fatalities + Injuries + StructuresDamaged + StructuresDestroyed',data = train).fit()
print(lrd10.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     11.50
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           5.21e-19
Time:                        08:16:29   Log-Likelihood:                -6942.3
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1169   BIC:                         1.396e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -18.35

In [ ]:
#removed major incident
lrd11 = smf.ols(formula = 'DaysElapsed ~ Latitude + Longitude + sunHour + CrewsInvolved + Engines + Fatalities + Injuries + StructuresDamaged + StructuresDestroyed',data = train).fit()
print(lrd11.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     12.70
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           1.97e-19
Time:                        08:16:30   Log-Likelihood:                -6942.6
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1170   BIC:                         1.396e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -14.6889    

In [ ]:
#removed longitude
lrd12 = smf.ols(formula = 'DaysElapsed ~ Latitude + sunHour + CrewsInvolved + Engines + Fatalities + Injuries + StructuresDamaged + StructuresDestroyed',data = train).fit()
print(lrd12.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.080
Method:                 Least Squares   F-statistic:                     13.85
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           2.38e-19
Time:                        08:16:31   Log-Likelihood:                -6944.3
No. Observations:                1180   AIC:                         1.391e+04
Df Residuals:                    1171   BIC:                         1.395e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              26.3916    

In [ ]:
cols12 = ['Latitude', 'sunHour', 'CrewsInvolved', 'Engines', 'Fatalities', 'Injuries', 'StructuresDamaged', 'StructuresDestroyed']
VIF(train, cols12)

Latitude                1.005380
sunHour                 1.083682
CrewsInvolved           3.832100
Engines                 3.632954
Fatalities             23.990329
Injuries                1.172631
StructuresDamaged       1.131569
StructuresDestroyed    23.817864
dtype: float64

In [ ]:
#removed Fatalities
lrd13 = smf.ols(formula = 'DaysElapsed ~ Latitude + sunHour + CrewsInvolved + Engines + Injuries + StructuresDamaged + StructuresDestroyed',data = train).fit()
print(lrd13.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     13.80
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           2.92e-17
Time:                        08:16:56   Log-Likelihood:                -6950.9
No. Observations:                1180   AIC:                         1.392e+04
Df Residuals:                    1172   BIC:                         1.396e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              38.6930    

In [ ]:
#removed structuresDestroyed
lrd14 = smf.ols(formula = 'DaysElapsed ~ Latitude + sunHour + CrewsInvolved + Engines + Injuries + StructuresDamaged',data = train).fit()
print(lrd14.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     16.08
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           7.74e-18
Time:                        08:16:59   Log-Likelihood:                -6951.0
No. Observations:                1180   AIC:                         1.392e+04
Df Residuals:                    1173   BIC:                         1.395e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            39.0650     40.32

In [ ]:
cols14 = ['Latitude', 'sunHour', 'CrewsInvolved', 'Engines', 'Injuries', 'StructuresDamaged']
VIF(train, cols14)

Latitude             1.001366
sunHour              1.066089
CrewsInvolved        3.807959
Engines              3.592990
Injuries             1.169590
StructuresDamaged    1.064174
dtype: float64

In [ ]:
#removed CrewsInvolved
lrd15 = smf.ols(formula = 'DaysElapsed ~ Latitude + sunHour + Engines + Injuries + StructuresDamaged',data = train).fit()
print(lrd15.summary())

                            OLS Regression Results                            
Dep. Variable:            DaysElapsed   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     16.71
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           5.68e-16
Time:                        08:17:08   Log-Likelihood:                -6957.1
No. Observations:                1180   AIC:                         1.393e+04
Df Residuals:                    1174   BIC:                         1.396e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            37.2490     40.51

In [ ]:
def rmse(model, df_test, dependent_var):
    y_test = df_test[dependent_var]
    y_pred = model.predict(df_test)
    se = (y_test - y_pred) ** 2
    mse = np.mean(se)
    return np.sqrt(mse)

In [ ]:
lrd_rmse = rmse(lrd14, test, 'DaysElapsed')
lrd_rmse

88.04418769930962

In [ ]:
max(fire6["DaysElapsed"])

465

In [ ]:
lrb1 = smf.ols(formula = 'AcresBurned ~ Latitude + Longitude + MajorIncident + AverageTempF + uvIndex + sunHour + totalSnow_cm + AirTankers + CrewsInvolved + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresEvacuated + StructuresThreatened + WaterTenders',data = train).fit()
print(lrb1.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     69.21
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          1.65e-175
Time:                        08:17:32   Log-Likelihood:                -13869.
No. Observations:                1180   AIC:                         2.778e+04
Df Residuals:                    1160   BIC:                         2.788e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -2.514e+

In [ ]:
#remove airtankers
lrb2 = smf.ols(formula = 'AcresBurned ~ Latitude + Longitude + MajorIncident + AverageTempF + uvIndex + sunHour + totalSnow_cm + CrewsInvolved + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresEvacuated + StructuresThreatened + WaterTenders',data = train).fit()
print(lrb2.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     73.12
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          1.92e-176
Time:                        08:17:33   Log-Likelihood:                -13869.
No. Observations:                1180   AIC:                         2.778e+04
Df Residuals:                    1161   BIC:                         2.787e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -2.494e+

In [ ]:
#remove total snow
lrb2 = smf.ols(formula = 'AcresBurned ~ Latitude + Longitude + MajorIncident + AverageTempF + uvIndex + sunHour + CrewsInvolved + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresEvacuated + StructuresThreatened + WaterTenders',data = train).fit()
print(lrb2.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     77.49
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          2.17e-177
Time:                        08:17:34   Log-Likelihood:                -13869.
No. Observations:                1180   AIC:                         2.777e+04
Df Residuals:                    1162   BIC:                         2.787e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -2.504e+

In [ ]:
#remove crews involved
lrb3 = smf.ols(formula = 'AcresBurned ~ Latitude + Longitude + MajorIncident + AverageTempF + uvIndex + sunHour + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresEvacuated + StructuresThreatened + WaterTenders',data = train).fit()
print(lrb3.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     82.34
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          2.97e-178
Time:                        08:17:37   Log-Likelihood:                -13870.
No. Observations:                1180   AIC:                         2.777e+04
Df Residuals:                    1163   BIC:                         2.786e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -2.441e+

In [ ]:
#remove longitude
lrb4 = smf.ols(formula = 'AcresBurned ~ Latitude + MajorIncident + AverageTempF + uvIndex + sunHour + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresEvacuated + StructuresThreatened + WaterTenders',data = train).fit()
print(lrb4.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     87.89
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          3.24e-179
Time:                        08:17:37   Log-Likelihood:                -13870.
No. Observations:                1180   AIC:                         2.777e+04
Df Residuals:                    1164   BIC:                         2.785e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -2.2e+

In [ ]:
#remove water tenders
lrb5 = smf.ols(formula = 'AcresBurned ~ Latitude + MajorIncident + AverageTempF + uvIndex + sunHour + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresEvacuated + StructuresThreatened',data = train).fit()
print(lrb5.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     94.10
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          5.27e-180
Time:                        08:17:38   Log-Likelihood:                -13870.
No. Observations:                1180   AIC:                         2.777e+04
Df Residuals:                    1165   BIC:                         2.785e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -2.212e+

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1755: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


In [ ]:
#remove structures evac
lrb6 = smf.ols(formula = 'AcresBurned ~ Latitude + MajorIncident + AverageTempF + uvIndex + sunHour + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened',data = train).fit()
print(lrb6.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     94.10
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          5.27e-180
Time:                        08:17:39   Log-Likelihood:                -13870.
No. Observations:                1180   AIC:                         2.777e+04
Df Residuals:                    1165   BIC:                         2.785e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -2.212e+

In [ ]:
#remove latitude
lrb7 = smf.ols(formula = 'AcresBurned ~ MajorIncident + AverageTempF + uvIndex + sunHour + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened',data = train).fit()
print(lrb7.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     101.2
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          9.00e-181
Time:                        08:17:39   Log-Likelihood:                -13871.
No. Observations:                1180   AIC:                         2.777e+04
Df Residuals:                    1166   BIC:                         2.784e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -6591.41

In [ ]:
#remove avg temp
lrb7 = smf.ols(formula = 'AcresBurned ~ MajorIncident + uvIndex + sunHour + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened',data = train).fit()
print(lrb7.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     109.7
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          9.61e-182
Time:                        08:17:40   Log-Likelihood:                -13871.
No. Observations:                1180   AIC:                         2.777e+04
Df Residuals:                    1167   BIC:                         2.783e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -2594.91

In [ ]:
#remove uv index
lrb8 = smf.ols(formula = 'AcresBurned ~ MajorIncident + sunHour + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened',data = train).fit()
print(lrb8.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.526
Method:                 Least Squares   F-statistic:                     119.8
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          9.71e-183
Time:                        08:17:40   Log-Likelihood:                -13871.
No. Observations:                1180   AIC:                         2.777e+04
Df Residuals:                    1168   BIC:                         2.783e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -3648.09

In [ ]:
#remove sun hour
lrb9 = smf.ols(formula = 'AcresBurned ~ MajorIncident + Dozers + Engines + Fatalities + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened',data = train).fit()
print(lrb9.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.526
Method:                 Least Squares   F-statistic:                     131.7
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          1.24e-183
Time:                        08:17:41   Log-Likelihood:                -13871.
No. Observations:                1180   AIC:                         2.776e+04
Df Residuals:                    1169   BIC:                         2.782e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2052.12

In [ ]:
cols9 = ['Dozers', 'Engines', 'Fatalities', 'Helicopters','Injuries','PersonnelInvolved', 'StructuresDamaged', 'StructuresDestroyed', 'StructuresThreatened']
VIF(train, cols9)

Dozers                   8.618963
Engines                 16.673500
Fatalities              24.389063
Helicopters              3.139410
Injuries                 1.346007
PersonnelInvolved       15.355505
StructuresDamaged        1.079809
StructuresDestroyed     23.994830
StructuresThreatened     1.256405
dtype: float64

In [ ]:
#remove fatalities
lrb10 = smf.ols(formula = 'AcresBurned ~ MajorIncident + Dozers + Engines + Helicopters + Injuries + PersonnelInvolved + StructuresDamaged + StructuresDestroyed + StructuresThreatened',data = train).fit()
print(lrb10.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                     129.5
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          8.93e-169
Time:                        08:17:45   Log-Likelihood:                -13909.
No. Observations:                1180   AIC:                         2.784e+04
Df Residuals:                    1170   BIC:                         2.789e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2075.30

In [ ]:
cols10 = ['Dozers', 'Engines', 'Helicopters','Injuries','PersonnelInvolved', 'StructuresDamaged', 'StructuresDestroyed', 'StructuresThreatened']
VIF(train, cols10)

Dozers                   8.404671
Engines                 16.598092
Helicopters              3.125859
Injuries                 1.340871
PersonnelInvolved       15.350473
StructuresDamaged        1.062333
StructuresDestroyed      1.091105
StructuresThreatened     1.255945
dtype: float64

In [ ]:
#remove personnel
lrb11 = smf.ols(formula = 'AcresBurned ~ MajorIncident + Dozers + Engines + Helicopters + Injuries + StructuresDamaged + StructuresDestroyed + StructuresThreatened',data = train).fit()
print(lrb11.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.491
Method:                 Least Squares   F-statistic:                     143.3
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          1.09e-167
Time:                        08:17:49   Log-Likelihood:                -13914.
No. Observations:                1180   AIC:                         2.785e+04
Df Residuals:                    1171   BIC:                         2.789e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2060.61

In [ ]:
cols11 = ['Dozers', 'Engines', 'Helicopters','Injuries', 'StructuresDamaged', 'StructuresDestroyed', 'StructuresThreatened']
VIF(train, cols11)

Dozers                  8.205179
Engines                 6.691949
Helicopters             3.080875
Injuries                1.315708
StructuresDamaged       1.061595
StructuresDestroyed     1.081384
StructuresThreatened    1.255529
dtype: float64

In [ ]:
#remove engines
lrb12 = smf.ols(formula = 'AcresBurned ~ MajorIncident + Dozers + Helicopters + Injuries + StructuresDamaged + StructuresDestroyed + StructuresThreatened',data = train).fit()
print(lrb12.summary())

                            OLS Regression Results                            
Dep. Variable:            AcresBurned   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     149.5
Date:                Tue, 07 Dec 2021   Prob (F-statistic):          1.53e-157
Time:                        08:17:52   Log-Likelihood:                -13940.
No. Observations:                1180   AIC:                         2.790e+04
Df Residuals:                    1172   BIC:                         2.794e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2036.52

In [ ]:
cols12 = ['Dozers',  'Helicopters','Injuries', 'StructuresDamaged', 'StructuresDestroyed', 'StructuresThreatened']
VIF(train, cols12)

Dozers                  3.286053
Helicopters             3.077766
Injuries                1.120862
StructuresDamaged       1.058949
StructuresDestroyed     1.062296
StructuresThreatened    1.255142
dtype: float64

In [ ]:
lrb_rmse = rmse(lrb12, test, 'AcresBurned')
lrb_rmse

65317.40707595

In [ ]:
np.mean(test['AcresBurned'])

12423.756756756757

In [ ]:
train["MajorIncident"] = train['MajorIncident'].astype(int)
test["MajorIncident"] = test['MajorIncident'].astype(int)

In [ ]:
lc1 = smf.logit(formula = 'MajorIncident ~ AverageTempF + uvIndex + Latitude + Longitude + sunHour + totalSnow_cm',
               data = train).fit()

print(lc1.summary())

Optimization terminated successfully.
         Current function value: 0.563815
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:          MajorIncident   No. Observations:                 1180
Model:                          Logit   Df Residuals:                     1173
Method:                           MLE   Df Model:                            6
Date:                Tue, 07 Dec 2021   Pseudo R-squ.:                 0.02824
Time:                        08:19:14   Log-Likelihood:                -665.30
converged:                       True   LL-Null:                       -684.64
Covariance Type:            nonrobust   LLR p-value:                 8.309e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -24.9036      6.116     -4.072      0.000     -36.890     -12.917
AverageTempF     0.0107

In [ ]:
fire_no0.groupby('MajorIncident').count()

,Latitude,Longitude,Started,Extinguished,Days Elapsed,AcresBurned,Average Temp F,uvIndex,sunHour,totalSnow_cm,AirTankers,CalFireIncident,Counties,CrewsInvolved,Dozers,Engines,Fatalities,Helicopters,Injuries,PersonnelInvolved,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,WaterTenders,DaysElapsed,AverageTempF
MajorIncident,,,,,,,,,,,,,,,,,,,,,,,,,,,
False,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922,922
True,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385,385


In [ ]:
ytest = test['MajorIncident']

yprob = lc1.predict(test)
ypred = pd.Series([1 if x > 0.40 else 0 for x in yprob], index=yprob.index)

In [ ]:
cm = confusion_matrix(ytest, ypred)
print ("Confusion Matrix : \n", cm) 

Confusion Matrix : 
 [[208   5]
 [ 80   3]]


In [ ]:
acc = (cm.ravel()[0]+cm.ravel()[3])/sum(cm.ravel())
acc

0.7128378378378378

In [ ]:
y_pred = ypred.astype('bool').to_numpy()
frame = {'Index':indices, 'Incident_LinLog':y_pred}
result = pd.DataFrame(frame)
result.head()

,Index,Incident_LinLog
0,1125,False
1,184,False
2,1226,False
3,67,False
4,220,False


In [ ]:
result.to_csv('LogisticPredictions.csv', index=False)